# Segment Real Time Data + Personalize - Lab 5 (optional)

In this module you are going to be adding the ability to maintain a real-time dataset that represents the latest user behavior for users of the Retail Demo Store.  You will then connect that dataset to the Personalize dataset groups that you built in the first part of the workshop. This will enable your Personalize models to be kept up to date with the latest events your users are performing.

This workshop will use the Segment analytics.js library to collect real-time data from the Retail Demo Store, and then feed that event data into the Segment platform, where it can be routed directly to a Personalize Event Tracker, and then used to maintain the latest behavioral data for your personalization user-item interaction data.

Recommended Time: 45 Minutes

## Prerequisites

In order to complete this workshop, you will need to complete the Labs 1-4 in this directory.  You will also need a Segment workspace.  If you are doing this workshop as part of a live workshop event, ask your moderator how to set up a Segment workspace.  If you are running this workshop on your own, you can [click here](https://segment.com) to create a Segment account.  We do not recommend using your production Segment workspace for this workshop.

## Segment Platform Overview
Segment is a customer data platform (CDP) that helps you collect, clean, and control your customer data. Segment provides several types of Sources which you can use to collect your data, and which you can choose from based on the needs of your app or site. For websites, you can use a javascript library to collect data. If you have a mobile app, you can embed one of Segment’s Mobile SDKs, and if you’d like to create messages directly on a server (if you have, for example a dedicated .NET server that processes payments), Segment has several server-based libraries that you can embed directly into your backend code. With Segment, you can also use cloud-sources to import data about your app or site from other tools like Zendesk or Salesforce, to enrich the data sent through Segment. By using Segment to decouple data collection from data use, you can create a centralized data supply chain based on organized and modular data.

![Segment Overview](images/segment/segment_overview.png)

## Setup

Segment uses *sources* as a way to organize data inputs into the platform.  Configuring a source will allow you to collect real-time event data from the Retail Demo Store user interface, and pass that information to Segment.  You need to be signed into your Segment workspace to begin this process.  Once you are signed in to the Segment console (https://app.segment.com), click on your workspace, and then ‘Connections’ in the upper left hand corner of the screen. Then, click ‘Add Source’.

![Segment Source Setup](images/segment/segment-step-1.png)

Select the ‘javascript’ source type.

![Segment Source Catalog](images/segment/segment-step-2.png)

And click ‘Add Source’.

![Segment Source Setup Add JS Source](images/segment/segment-step-3.png)

Give your source a name.  We recommend ‘retail-demo-store.’

![Segment Source Setup](images/segment/segment-step-4.png)

The next screen confirms that your source was successfully created.

![Segment Source Setup](images/segment/segment-step-5.png)

Now that you are here, set the write key for your new source in the environment variable below.  

Copy the string in the `analytics.load(“...”)` section shown above, and paste it into the variable in the cell below.  

You will need this in a few minutes, when you enable Segment events collection in the Retail Demo Store.

Make sure you run the cell after you paste the key.

In [ ]:
import boto3

# Enter your Segment write key here, from the above step

segment_write_key = "YOUR_SEGMENT_WRITE_KEY_GOES_HERE"


Now that you have a working source, let’s wire up our event data to Amazon Personalize.

## Configure the Segment Personalize Destination

Segment uses Destinations to route real-time event data to a data consumer application.  In this case, you will be using the Segment Personalize destination.  This destination will take real-time events from Segment, pass them through an AWS Lambda function, and then into the user-item interactions dataset in your Retail Demo Store.

Click ‘Connections’ in the upper left corner of the screen, and then the ‘Add Destination’ button.

![Segment Destination Setup](images/segment/segment-step-6.png)

In the Segment destinations catalog, type ‘personalize’ into the search text box in the upper left corner of the screen.  You will see the Amazon Personalize destination appear in the search results.  Click the tile for the destination.

![Segment Destination Setup](images/segment/segment-step-7.png)

Then, click the ‘Configure Amazon Personalize’ button.

![Segment Destination Setup](images/segment/segment-step-8.png)

Select your source from the earlier part of this process; this screen should show the source you created in the steps above.  

![Segment Destination Setup](images/segment/segment-step-9.png)

Then, click the ‘Confirm Source’ button.

![Segment Destination Setup](images/segment/segment-step-10.png)

At this point, you are ready to configure the AWS Lambda function that the destination will need to process events from the Retail Demo Store.

![Segment Destination Setup](images/segment/segment-step-11.png)

The Retail Demo Store deployment in your AWS account will have a pre-configured Lambda function that you can use to configure the destination.  This function is deployed to the Retail Demo store during the environment setup steps at the beginning of the workshop.

Segment uses a Lambda function as a way to allow you to customize the event names and properties that are sent to Personalize for training purposes.  The Cloud Formation Template and code for this Lambda is available for your use, if you would like to use this pattern in your own applications.

Log in to your AWS Console, and select Lambda under the Services finder in the top left corner of the screen.  You will see a screen that looks like this:

![Segment Destination Setup](images/segment/segment-step-12.png)

Find the SegmentPersonalizeEventsDestination and click on it.  Keep this tab or window open as you will need it in a few steps when you configure Amazon Personalize.

![Segment Destination Setup](images/segment/segment-step-13.png)

At the top of the screen, you will see the ARN for the Lambda function.  Click the copy link shown above to copy the ARN to the clipboard, then go back to the Segment console for your Personalize destination, and click on Lambda **.

![Segment Destination Setup](images/segment/segment-step-14.png)

On the next screen, paste in the ARN for the Lambda.  And click the Save button.

![Segment Destination Setup](images/segment/segment-step-15.png)

Next, you will give Segment permission to call your Lambda from their AWS account.  Click the Role Address** link.

![Segment Destination Setup](images/segment/segment-step-16.png)

Next, go to your AWS Console tab or window, and select IAM from Services, and then click on Roles. In the search text box, search for Segment.

![Segment Destination Setup](images/segment/segment-step-17.png)

Select the SegmentCrossAccountLambda role, and copy the Role ARN.  You can easily copy the whole string by clicking the copy button to the right of the ARN as shown in the screen shot.

![Segment Destination Setup](images/segment/segment-step-18.png)

Go back to the Segment console destination configuration tab or window, and paste in the ARN you just copied and click Save.

![Segment Destination Setup](images/segment/segment-step-19.png)

Next, you need to set the region for the Lambda that is deployed in your account.

If you are running this in an AWS managed workshop, ask your event admins for the region in which you are running the workshop.  If you are running the workshop in us-west-2, there is no need to change this setting.  Otherwise, use the region in which the workshop is deployed, otherwise Segment will not be able to invoke your Lambda.

![Segment Destination Setup](images/segment/segment-step-22.png)


![Segment Destination Setup](images/segment/segment-step-23.png)

Segment will call the Lambda using the Role SegmentCrossAccountLambdaExecutionRole. Currently this IAM role has externalId 123456789. You need to replace it using autogenerated externalId in Segment. To update role go to IAM from AWS console & search for SegmentCrossAccountLambdaExecutionRole. This role is used while Segment subsequently calling Amazon Personalize. Replace the externalId with the one randomly generated as shown in the example below.

![Segment Destination Setup](images/segment/segment-step-22-1.png)

Finally, click the slider button at the top of the screen to enable the destination.  You must enable the destination or events will not be sent to Personalize in the following steps.

![Segment Destination Setup](images/segment/segment-step-24.png)

## Configure Lambda Parameters and Review Code

Before the destination can send events to your Amazon Personalize event tracker, you will need to tell the destination lambda where to send the events.  It looks for an environment variable called 'personalize_tracking_id'.

Let's set that.  Run the following cell to look up the relevant Amazon Personalize tracker from the Personalize workbook.

We can then set the appropriate value in the destination Lambda.

In [ ]:
ssm = boto3.client('ssm')

# First, let's look up the appropriate tracking string
response = ssm.get_parameter(
    Name='/retaildemostore/personalize/event-tracker-id'
)

tracking_id = response['Parameter']['Value']

print(tracking_id)

Go to your AWS console tab or window, and select Lambda from the Services menu.

Find the SegmentPersonalizeEventsDestination, and click on it in the list.

![Segment Events Destination Lambda](images/segment/segment-find-lambda-function.png)

![Segment Events Destination Lambda](images/segment/segment-events-lambda.png)

Then, scroll down to the parameters section.

![Segment Events Destination Lambda](images/segment/segment-events-lambda-params.png)

The tracking parameter should be set to the tracker from the first workbook.  

Click the edit button, then paste in the tracking ID from the cell above, and click the Redeploy button at the top of the screen.

Take some time to look at the code that this Lambda uses to send events to Personalize.  You can use this code in your own deployment, however you may need to change the event parameters sent to Amazon Personalize depending on the dataset you set up.

```python
def lambda_handler(event, context):    
    # In high volume applications, remove this code.
    logger.debug("Got event: " + json.dumps(event))

    # Segment will invoke your function once per event type you have configured
    # in the Personalize destination in Segment.
    try:
        if ('anonymousId' in event or 'userId' in event and 'properties' in event):
            # Make sure this event contains an itemId since this is required for the Retail Demo Store
            # dataset - you can also check for specific event names here if needed, and only pass the ones
            # that you want to use in the training dataset
            if (not 'productId' in event['properties']):
                logger.debug("Got event with no productId, discarding.")
                return

            logger.debug("Calling putEvents()")
            # Function parameters for put_events call.
            params = {
                'trackingId': personalize_tracking_id,
                'sessionId': event['anonymousId']
            }

            # If a user is signed in, we'll get a userId. Otherwise for anonymous 
            # sessions, we will not have a userId. We still want to call put_events
            # in both cases. Once the user identifies themsevles for the session, 
            # subsequent events will have the userId for the same session and 
            # Personalize will be able to connect prior anonymous to that user.
            if event.get('userId'):
                params['userId'] = event['userId']

            # YOU WILL NEED TO MODIFY THIS PART TO MATCH THE EVENT PROPS
            # THAT COME FROM YOUR EVENTS
            # 
            # Personalize needs the event identifier
            # that was used to train the model. In this case, we're using the 
            # product's productId passed through Segment to represent the itemId.
            #
            properties = { 'itemId': event['properties']['productId'] }

            # Build the event that we're sending to Personalize.  Note that Personalize
            # expects a specific event format
            personalize_event = {
                'eventId': event['messageId'],
                'sentAt': int(dp.parse(event['timestamp']).strftime('%s')),
                'eventType': event['event'],
                'properties': json.dumps(properties)
            }

            params['eventList'] = [ personalize_event ]

            logger.debug('put_events parameters: {}'.format(json.dumps(params, indent = 2)))
            # Call put_events
            response = personalize_events.put_events(**params)
        else:
            logger.debug("Segment event does not contain required fields (anonymousId and sku)")
```


## Validate that Real-Time Events are Flowing from the Retail Demo Store

You are now ready to send live events to Personalize from the Retail Demo Store.  In order to do this, you will need to enable the Segment client side integration with the Retail Demo Store.  Segment provides a variety of ways to collect real time events, and a full discussion of how this works is beyond the scope of this document, however the Retail Demo Store represents a fairly typical deployment for most web applications, in that it uses the Segment analytics.js library, loaded via the Segment CDN, to inject their code into the web application.  

Because the Retail Demo Store is a Vue.js application, this code is loaded in the head tag of index.html file:

```html
  <head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <link rel="apple-touch-icon" sizes="180x180" href="<%= BASE_URL %>apple-touch-icon.png">
    <link rel="icon" type="image/png" sizes="32x32" href="<%= BASE_URL %>favicon-32x32.png">
    <link rel="icon" type="image/png" sizes="16x16" href="<%= BASE_URL %>favicon-16x16.png">
    <link rel="manifest" href="<%= BASE_URL %>site.webmanifest">
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
    <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.3.1/css/all.css" integrity="sha384-mzrmE5qonljUremFsqc01SB46JvROS7bZs3IO2EmfFsd15uHvIt+Y8vEf7N7fWAU" crossorigin="anonymous">
    <title>Retail Demo Store</title>
    <!-- This code is for the Segment integration - DO NOT REMOVE -->
    <%= VITE_SEGMENT_WRITE_KEY === 'NONE' ? '' : `<script>\r\n  !function(){var analytics=window.analytics=window.analytics||[];if(!analytics.initialize)if(analytics.invoked)window.console&&console.error&&console.error(\"Segment snippet included twice.\");else{analytics.invoked=!0;analytics.methods=[\"trackSubmit\",\"trackClick\",\"trackLink\",\"trackForm\",\"pageview\",\"identify\",\"reset\",\"group\",\"track\",\"ready\",\"alias\",\"debug\",\"page\",\"once\",\"off\",\"on\",\"addSourceMiddleware\",\"addIntegrationMiddleware\",\"setAnonymousId\",\"addDestinationMiddleware\"];analytics.factory=function(e){return function(){var t=Array.prototype.slice.call(arguments);t.unshift(e);analytics.push(t);return analytics}};for(var e=0;e<analytics.methods.length;e++){var key=analytics.methods[e];analytics[key]=analytics.factory(key)}analytics.load=function(key,e){var t=document.createElement(\"script\");t.type=\"text\/javascript\";t.async=!0;t.src=\"https:\/\/cdn.segment.com\/analytics.js\/v1\/\" + key + \"\/analytics.min.js\";var n=document.getElementsByTagName(\"script\")[0];n.parentNode.insertBefore(t,n);analytics._loadOptions=e};analytics.SNIPPET_VERSION=\"4.13.1\";\r\n  analytics.load(\"${VITE_SEGMENT_WRITE_KEY}\");\r\n analytics.page();\r\n }}();\r\n<\/script>` %>
  </head>
```

This ensures that the Segment library is available at all times for each user action in the user interface of the Retail Demo Store.  Each time a user performs an action in the Retail Demo Store user interface (such as viewing a product, adding a product to a cart, searching, etc.) an event is sent to Segment with the properties associated with that user’s action.  Note that this code is only loaded if an environment variable is set to define the write key for the Segment source which will accept events from the Retail Demo Store.  This will become important in a moment.

```javascript
        let eventProperties = {
            productId: product.id,
            name: product.name,
            category: product.category,
            image: product.image,
            feature: feature,
            experimentCorrelationId: experimentCorrelationId,
            price: +product.price.toFixed(2)
        };

        if (this.segmentEnabled()) {
            window.analytics.track('View', eventProperties);
        }
```

This allows you to collect data that is relevant to any tool that might need to be kept updated with the latest user behavior.  

## Sending Real-Time Events

Since you have already connected Segment to Personalize, let’s test this data path by triggering events from the Retail Demo Store user interface which is deployed in your AWS account.

Run the following code to set the SSM parameter that holds the Segment write key.

In [ ]:
# Set the Segment write key in the string below, and run this cell.  

# THIS IS ONLY REQUIRED IF YOU DID NOT SET THE SEGMENT WRITE KEY IN YOUR ORIGINAL DEPLOYMENT

import boto3

ssm = boto3.client('ssm')

if segment_write_key:
    response = ssm.put_parameter(
        Name='retaildemostore-segment-write-key',
        Value='{}'.format(segment_write_key),
        Type='String',
        Overwrite=True
    )

print(segment_write_key)

You now have an environment variable that will enable the Segment data collection library in the code in `index.html`.  All we need to do now, is force a re-deploy of the Retail Demo Store.  

To do that, go back to your AWS Console tab or window, and select Code Pipeline from the Services search.  Then, find the pipeline name that contains `WebUIPipeline` and click that link.

![Segment Destination Setup](images/segment/segment-step-25.png)

Then, select the ‘Release Change’ button, and confirm the release once the popup shows up.  You will see a confirmation that the pipeline is re-deploying the web ui for the Retail Demo Store.

![Segment Destination Setup](images/segment/segment-step-26.png)

This process should complete in a few minutes.  Once this is done, you will see the bottom tile confirm that your deployment has completed:

![Segment Destination Setup](images/segment/segment-step-27.png)

## Log in as a Retail Demo Store User

**IMPORTANT**

Once this is confirmed, tab back to the Retail Demo Store web UI, and refresh the screen to reload the user interface.  This will load the libraries you just deployed, and will allow your instance of the Retail Demo Store to send events to Segment.

In the Personalize workshop, you created an account for the Retail Demo Store.  If you are not logged in as that account, log in now.

Then, open another tab to the Segment console, and select ‘retail-demo-store’ under the Sources tab:

![Segment Destination Setup](images/segment/segment-step-28.png)

And then click on the Debugger tab.

![Segment Destination Setup](images/segment/segment-step-29.png)

Here you will be able to see live events coming from the Retail Demo Store this:

![Segment Destination Setup](images/segment/segment-step-30.png)

Go back to the Retail Demo Store web app tab, and click on some categories of products.  You should start to see events appear in the Segment debugger at this point.  This means that the actions you are performing as a user are now being sent to the training datasets in Personalize.

Now that you have events flowing from your web application to Personalize, you can keep the Personalize data sets up to date, and re-train them as required by your application use cases.

You can finish the workshop here, or learn more about what you can do with Segment’s Customer Data Platform (Personas) and Amazon Personalize in the next workbook:

[Segment CDP Workbook](../6-CustomerDataPlatforms/6.1-Segment.ipynb)